# Alberi decisionali

In [ ]:
# Load libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation



In [ ]:
# load dataset
df = pd.read_csv("diabetes.csv")


In [ ]:
df.head()


In [ ]:
#split dataset in features and target variable
feature_cols = ['Pregnancies', 'Insulin', 'BMI', 'Age','Glucose','BloodPressure','DiabetesPedigreeFunction']
X = df[feature_cols] # Features
y = df.Outcome # Target variable


In [ ]:
X= df.iloc[:,:8] #feature
y=df['Outcome'] # target

In [ ]:
df.dtypes

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test


## Data exploration

In [ ]:
plt.bar(df["Outcome"].unique(), df["Outcome"].value_counts())
plt.xlabel("Outcome")
plt.ylabel("Frequenza")
plt.title("Distribuzione di \"Outcome\"")
plt.xticks([0, 1])
plt.show()

In [ ]:
df['glucose_group'] = pd.cut(df['Glucose'], bins=[0, 125, float('inf')], labels=['Low (<= 120)', 'High (> 120)'])
grouped = df.groupby(['glucose_group', 'Outcome']).size().unstack(fill_value=0)
grouped.plot(kind='bar', stacked=True)
plt.title('Grafico a barre stacked di Glucose Group e Outcome')
plt.xlabel('Glucose Group')
plt.ylabel('Count')
plt.legend(title='Outcome', labels=['0', '1'])
plt.show()

## Creazione del training set e del test set

In [ ]:
feature_names = set(df.columns) - set(["Outcome"])
list(feature_names)


In [ ]:
df=df.drop(['glucose_group'], axis=1)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

feature_names = list(set(df.columns) - set(["Outcome"]))

target_name = "Outcome"

X_train, X_test, y_train, y_test = train_test_split(df[feature_names], df[target_name], test_size=0.3, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(df[feature_names], df[target_name], test_size=0.3, random_state=42)

In [ ]:
print(f"Numero di record nel training set: {X_train.shape[0]}")
print(f"Numero di record nel test set: {X_test.shape[0]}")

## Modello di baseline

Vediamo nel training set, e nel test set, quante persone sono sopravvissute

In [ ]:
y_train.value_counts() / y_train.count()

In [ ]:
y_test.value_counts() / y_test.count()

Creiamo dunque un semplice classificatore che etichetta sempre negativamente le istanze

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# y_pred è un vettore di soli zero (predizioni del modello di baseline)
y_pred = np.zeros(y_test.shape)

cm = confusion_matrix(y_test, y_pred)

print("Confusion matrix:\n", cm)
print("\nAccuracy:", cm.diagonal().sum() / cm.sum())

## Let's make a better model

Come abbiamo mostrato anche in precedenza con il grafico a barre, la maggioranza dei pazienti con Outcome 1 ha livelli di glucosio maggiore di 120.

In [ ]:
# predizione del nuovo modello
y_pred = (X_test["Glucose"] > 120).astype(int)

cm = confusion_matrix(y_test, y_pred)

print("Confusion matrix:\n", cm)
print("\nAccuracy:", cm.diagonal().sum() / cm.sum())

Notiamo che le performance sono migliorate

## Costruiamo un albero decisionale


In [ ]:
X_train.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

model = DecisionTreeClassifier(random_state=42)
model = model.fit(X_train, y_train)

# Visualizza l'albero decisionale
fig, ax = plt.subplots(figsize=(150, 100))
plot_tree(model, filled=True, ax=ax)
plt.plot()

## Inferenza


In [ ]:
# predizione del nuovo modello
y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

print("Confusion matrix:\n", cm)
print("\nAccuracy:", cm.diagonal().sum() / cm.sum())

##  Costruiamo un albero di decisione  più efficiente ed efficace

In [ ]:
prunedModel = DecisionTreeClassifier(random_state=42,max_depth=5, min_samples_split=2)
prunedModel = prunedModel.fit(X_train, y_train)

# Visualizza l'albero decisionale
fig, ax = plt.subplots(figsize=(150, 100))
plot_tree(prunedModel, filled=True, ax=ax)
plt.plot()

**Migliore interpretabilità**

Per una visualizzazione più comprensibile per l'utente finale creiamo una figura che contenga i percorsi decisionali con i nomi delle variabili

In [ ]:
pip install graphviz

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

# Esporta l'albero di decisione in formato dot
dot_data = export_graphviz(
    prunedModel,
    out_file=None,  # Non salvare su file, ma generare come stringa
    feature_names=X_train.columns,
    class_names=["yes", "no"],
    filled=True,
    rounded=True,
    special_characters=True
)

# Usa graphviz per visualizzare l'albero
graph = graphviz.Source(dot_data)
graph.render("diabetes_decision_tree", format="png", cleanup=True)  # Salva l'immagine come PNG
graph.view()  # Mostra l'immagine dell'albero


In [ ]:
# predizione del nuovo modello
y_pred = prunedModel.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

print("Confusion matrix of the pruned model:\n", cm)
print("\nAccuracy of the pruned model:", cm.diagonal().sum() / cm.sum())

## Parametro di complessità

Vediamo come varia l'accuratezza dell'albero decizionale in funzione della complessità dell'albero

https://scikit-learn.org/1.5/modules/tree.html#minimal-cost-complexity-pruning

In [ ]:
path = model.cost_complexity_pruning_path(X_train, y_train)

ccp_alphas = path.ccp_alphas

In [ ]:
# Addestra l'albero decisionale con diversi valori di complessità
train_accuracy = []
test_accuracy = []
for complexity in ccp_alphas:
    clf = DecisionTreeClassifier(max_depth=5, ccp_alpha=complexity)
    clf.fit(X_train, y_train)
    train_accuracy.append(clf.score(X_train, y_train))
    test_accuracy.append(clf.score(X_test, y_test))

# Plotta l'accuratezza del modello in funzione del parametro di complessità
plt.plot(ccp_alphas, train_accuracy, label='Training Accuracy')
plt.plot(ccp_alphas, test_accuracy, label='Test Accuracy')
plt.xlabel('Complexity Parameter')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Complexity Parameter')
plt.xscale('log')
plt.legend()
plt.show()

## Assignment

1. Disegnare gli alberi tagliati in accordo al parametro di complessità
2. Provate a creare  altri alberi modificando i parametri nel costruttore DecisionTreeClassifier(). Ad esempio, cosa succede se si cambiano i criteri di splitting? Come varia l'accuratezza?